## 하이퍼 파라미터 튜닝
- XGBoost나 LightGBM에 Grid Search를 적용할 경우 하이퍼 파라미터 최적화 시간이 기하급수적으로 늘어남. 그래서 실무의 대용량 학습 데이터에 XGBoost나 LightGBM의 하이퍼 파라미터 튜닝 시에는 다른 방식을 적용 
> 베이지안 최적화 기법: 목적 함수 식을 제대로 알 수 없는 블랙 박스 형태의 함수에서 최대 또는 최소 함수 반환 값을 만드는 최적 입력값을, 가능한 적은 시도를 통해 빠르고 효과적으로 찾아주는 방식, 베이지안 확률에 기반을 두고 있는 기법. 

- 두 가지 구성 요소: 대체 모델, 획득 함수
=> 획득 함수로부터 최적 함수를 예측할 수 있는 입력값을 추천 받은 뒤 이를 기반으로 최적 함수 모델 개선, 
획득 함수는 이 개선된 대체 모델을 기반으로 최적 입력값 계산, 이 과정 반복(최적 값 기반으로 모델 갱신, 다시 최적 함수 예측 추정)

- 관련 파이썬 패키지: HyperOpt, Bayesian Optimization, Optuna

In [1]:
from hyperopt import hp

# -10 ~ 10까지 1 간격을 가지는 입력 변수 x와 -15 ~ 15까지 1 간격으로 입력 변수 y 설정
search_space = {'x':hp.quniform('x', -10,10,1), 'y':hp.quniform('y',-15,15,1)}

In [2]:
from hyperopt import STATUS_OK

# 목적함수 생성, 변숫값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환

def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    
    return retval

In [4]:
from hyperopt import fmin, tpe, Trials
import numpy as np

trial_val = Trials()

best_01 = fmin(fn = objective_func, space=search_space, algo=tpe.suggest, max_evals=5, 
              trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_01)

100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 429.33trial/s, best loss: -224.0]
best: {'x': -4.0, 'y': 12.0}


In [5]:
best_02 = fmin(fn = objective_func, space=search_space, algo=tpe.suggest, max_evals=20, 
              trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 302.23trial/s, best loss: -296.0]
best: {'x': 2.0, 'y': 15.0}


In [6]:
# fmin()에 인자로 들어가는 Trials객체의 result속성에 파이썬 리스트로 목적 함수 "반환 값들"이 저장됨
# {'loss':함수 반환값, 'status':반환 상태값} 과 같은 딕셔너리 형태
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}]


In [7]:
# Trials객체의 val속성에는 개별 수행 시마다 "입력된" 값 리스트 저장
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, -6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0]}


## XGBoost 하이퍼 파라미터 튜닝: 유방암 데이터 예제

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def get_new_feature_df(old_feature_name_df): 
    feature_dup_df = pd.DataFrame(data = old_feature_name_df.groupby('column_name').cumcount(), columns = ['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1]) if x[1]>0 \
                                                                                             else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

import xgboost as xgb
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
features = dataset.data
labels = dataset.target

cancer_df = pd.DataFrame(data = features, columns = dataset.feature_names)
cancer_df['target'] = labels
cancer_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0


In [10]:
X_features = cancer_df.iloc[:,:-1]
y_label = cancer_df.iloc[:,-1]

# 전체에서 80%는 학습용, 20%는 테스트용
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2, random_state=156)

# 위에서 만든 학습용을 다시 쪼개서 90%는 학습, 10%는 검증용
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=156)

In [11]:
from hyperopt import hp

# serach space: 입력 값, 하이퍼 파라미터에 대해서 검색 공간 설정, 여기서는 xgb의 하이퍼 파라미터
# max_depth, min_child_weight은 정수형이니까 hp.quniform, 그런데 실수형 값으로 입력되므로 정수형으로 형변환 필요
xgb_search_space = {'max_depth':hp.quniform('max_depth',5,20,1), 
                   'min_child_weight':hp.quniform('min_child_weight',1,2,1),
                   'learning_rate':hp.uniform('learning_rate',0.01,0.2),
                   'colsample_bytree':hp.uniform('coluniform_bytree',0.5,1)}

In [13]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# 정확도는 높을수록 더 좋은 수치이므로, -1을 곱해서 큰 정확도 값일수록 최소가 되도록 변환(fmin() 함수는 최솟값을 최적화하므로)
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']), 
                                                           min_child_weight=int(search_space['min_child_weight']),
                                                           learning_rate = search_space['learning_rate'],
                                                           colsample_bytree=search_space['colsample_bytree'],
                                                           eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    
    # accuracy는 cv=3 개수 만큼 roc-auc결과를 리스트로 가짐. 평균해서 반환하는데 -1을 곱해야 함.
    return {'loss':-1*np.mean(accuracy), 'status':STATUS_OK}

In [16]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best=fmin(fn=objective_func, 
          space=xgb_search_space,algo = tpe.suggest, max_evals=50, trials=trial_val, rstate=np.random.default_rng(seed=9))

print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [00:08<00:00,  5.57trial/s, best loss: -0.9670616939700244]
best: {'coluniform_bytree': 0.5424149213362504, 'learning_rate': 0.12601372924444681, 'max_depth': 17.0, 'min_child_weight': 2.0}
